In [1]:
import pandas as pd

# Load the CSV file
file_path = 'realadvisor.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()

,web-scraper-order,web-scraper-start-url,zeige-alles,zeige-alles-href,subpages,subpages-href,durchschnitt-raw,unten-oben-raw
0,1701727323-189,https://realadvisor.ch/de/immobilienpreise-pro...,NaN,NaN,Windisch,https://realadvisor.ch/de/immobilienpreise-pro...,CHF 7’246,CHF 5’701 – 10’928*
1,1701727324-1,https://realadvisor.ch/de/immobilienpreise-pro...,NaN,NaN,Wiliberg,https://realadvisor.ch/de/immobilienpreise-pro...,CHF 5’768,CHF 3’934 – 8’410*
2,1701727325-190,https://realadvisor.ch/de/immobilienpreise-pro...,NaN,NaN,Gränichen,https://realadvisor.ch/de/immobilienpreise-pro...,CHF 6’271,CHF 4’892 – 9’377*
3,1701727326-2,https://realadvisor.ch/de/immobilienpreise-pro...,NaN,NaN,Böbikon,https://realadvisor.ch/de/immobilienpreise-pro...,N / A,NaN
4,1701727327-191,https://realadvisor.ch/de/immobilienpreise-pro...,NaN,NaN,Bremgarten (AG),https://realadvisor.ch/de/immobilienpreise-pro...,CHF 7’772,CHF 5’559 – 10’657*


In [ ]:
# Data Cleaning

# Remove currency symbols and non-numeric characters from 'durchschnitt-raw'
data['durchschnitt'] = data['durchschnitt-raw'].str.extract(r'(\d+’\d+)').replace('’', '', regex=True).astype(float)

# Split 'unten-oben-raw' into two columns 'unten' and 'oben'
data[['unten', 'oben']] = data['unten-oben-raw'].str.extract(r'(\d+’\d+).*?(\d+’\d+)').replace('’', '', regex=True).astype(float)

# Keep only the relevant columns ('subpages', 'durchschnitt', 'unten', 'oben')
cleaned_data = data[['subpages', 'durchschnitt', 'unten', 'oben']]

# Display the cleaned data
cleaned_data.head()


In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='hostname',
                                         database='realadvisor',
                                         user='username',
                                         password='password')

    if connection.is_connected():
        cursor = connection.cursor()

        # Insert each row from the DataFrame
        for row in cleaned_data.itertuples():
            mysql_insert_query = """INSERT INTO property_data (location, average, range_low, range_high) 
                                    VALUES (%s, %s, %s, %s) """
            record = (row.subpages, row.durchschnitt, row.unten, row.oben)
            cursor.execute(mysql_insert_query, record)

        connection.commit()

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
